In [1]:
#实验 kernal-kmeans

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_wine

In [3]:
#1.加载数据
wine=load_wine()
scaler =StandardScaler()
x=scaler.fit_transform(wine.data)
y=wine.target

In [4]:
import numpy as np
#2.自定义k-means距离衡量标准
def euclidean_distance(one_sample,x):
    one_sample=one_sample.reshape(1,-1)
    distances=np.power(np.tile(one_sample,(x.shape[0],1))-x,2).sum(axis=1)
    return distances

In [5]:
#2.自定义k-means类
import random

In [6]:
def process(_x):
    '''
    映射到高维核空间
    :param data_point:
    :param data_noise:
    :return:
    '''
    Z = np.zeros([_x.shape[0], 3])
    # 二项式映射
    # X[:,0] = _x[:,0]**2
    # X[:, 1] = math.sqrt(2)*_x[:,0]*_x[:,1]
    # X[:,2] = _x[:,1]**2

    # 高斯核映射
    Z[:, 0] = np.exp(-(_x[:, 0] ** 2)) * np.exp(-(_x[:, 1] ** 2))
    Z[:, 1] = 2 * _x[:, 0] * _x[:, 1] * np.exp(-(_x[:, 0] ** 2)) * np.exp(-(_x[:, 1] ** 2))
    Z[:, 2] = 2 * _x[:, 0] ** 2 * _x[:, 1] ** 2 * np.exp(-(_x[:, 0] ** 2)) * np.exp(-(_x[:, 1] ** 2))

    return Z

In [7]:
class kmeans():
    """
    Kmeans聚类

    """
    def __init__(self,k=2,max_iterations=500, varepsilon=0.0001):
        """
        k:int 聚类数目
        max_iterations:int  最大迭代数目
        varepsilon:float 判断是否收敛，如果上一次的所有k个聚类中心与本次所有k个聚类中心都小于这个值，那么说明已经收敛
        """
        self.k=k
        self.max_iterations=max_iterations
        self.varepsilon=varepsilon
        random.seed(1)
    def init_random_centroids(self,x):
        #从所有样本中随机选取样本作为初始的聚类中心
        """
        输入x 即所需聚类的样本
        随机进行初始化，返回初始化聚类中心
        """
        n_samples,n_features=x.shape
        centroids=np.zeros((self.k,n_features))
        for i in range(self.k):
            centroid=x[random.choice(range(n_samples))]
            centroids[i]=centroid
        return centroids
    def _closet_centroid(self,sample,centoids):
        #返回距离该样本最近的中心索引
        """
        输入样本 sample,centoids中心坐标
        返回最近的中心的索引
        
        """
        distances=euclidean_distance(sample,centoids)
        closest_i=np.argmin(distances)

        return closest_i
    def create_clusters(self,centroids,x):
        """
        输入 x 样本
        centroids 中心坐标
        输出 聚类结果
        """
        clusters=[ [] for _ in range(self.k)]
        for sample_i,sample in enumerate(x):
            centroid_i=self._closet_centroid(sample,centroids)
            clusters[centroid_i].append(sample_i)
        return clusters
    def update_centroids(self,clusters,x):
        """
        更新中心点
        输入 clusters 聚类结果
        x 样本
        返回 新的中心点坐标
        """
        n_features=x.shape[1]
        centroids=np.zeros((self.k,n_features))
        for i ,cluster in enumerate(clusters):
            centroid=np.mean(x[cluster],axis=0)
            centroids[i]=centroid
        return centroids
    def get_cluster_labels(self,clusters,x):
        """
        输入 x 样本
        centroids 中心坐标
        输出 聚类结果标签
        """
        y_pred=np.zeros(x.shape[0])
        for cluster_i,cluster in enumerate(clusters):
            for sample_i in cluster:
                y_pred[sample_i]=cluster_i
        return y_pred

    def predict(self,x):
        """
        预测x对应的label
        """
        centroids=self.init_random_centroids(x)

        for _ in range(self.max_iterations):
            clusters=self.create_clusters(centroids,x)
            former_centroids=centroids
            centroids=self.update_centroids(clusters,x)
            diff=centroids-former_centroids
            if diff.any() <self.varepsilon:
                break
        return self.get_cluster_labels(clusters,x)



In [8]:
x=x.tolist()
x=np.array(x)

In [9]:
km=kmeans(k=3)
x=process(x)
y_pred=km.predict(x)
y[y==0]=-1
y[y==1]=-2
y[y==2]=-3

In [10]:
y

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2,
       -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3, -3,
       -3, -3, -3, -3, -3, -3, -3, -3])

In [11]:
y_pred[y_pred==0]=-1
y_pred[y_pred==2]=-2
y_pred[y_pred==1]=-3

In [12]:
y_pred

array([-2., -1., -1., -2., -1., -2., -2., -2., -2., -2., -2., -2., -2.,
       -2., -2., -1., -2., -2., -2., -3., -2., -2., -1., -1., -1., -1.,
       -1., -1., -2., -2., -2., -1., -1., -2., -1., -1., -1., -1., -1.,
       -2., -1., -3., -2., -2., -1., -2., -2., -2., -2., -2., -1., -2.,
       -2., -2., -2., -1., -2., -1., -2., -3., -3., -3., -2., -3., -3.,
       -3., -2., -3., -2., -3., -3., -2., -1., -1., -3., -2., -2., -2.,
       -3., -2., -3., -1., -3., -2., -2., -3., -3., -2., -2., -3., -3.,
       -3., -3., -1., -2., -3., -2., -3., -3., -2., -3., -3., -1., -3.,
       -3., -1., -3., -1., -3., -2., -2., -1., -2., -2., -3., -2., -3.,
       -3., -2., -2., -2., -2., -2., -2., -2., -3., -3., -2., -3., -2.,
       -3., -1., -1., -2., -3., -1., -2., -2., -3., -1., -1., -1., -3.,
       -2., -2., -3., -2., -2., -3., -2., -3., -1., -1., -3., -3., -2.,
       -2., -1., -2., -1., -2., -3., -1., -2., -3., -2., -3., -1., -1.,
       -2., -2., -1., -2., -2., -3., -2., -1., -2.])

In [13]:
#3.对数据集聚类 计算吻合度
acc=accuracy_score(y,y_pred)
print('吻合度{:.2f}'.format(acc))

吻合度0.36
